In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import matplotlib.pyplot as plt


In [11]:
df = pd.read_csv('df_all_fights.csv')
df.columns = df.columns.str.strip()


In [13]:
num_cols = [
    'strike_accuracy_diff',
    'total_strike_accuracy_diff',
    'knockdowns_diff',
    'takedown_accuracy_diff',
    'takedowns_diff',
    'submission_attempts_diff'
]
cat_cols = ['weight_class', 'method']

df_clean = df[num_cols + cat_cols].dropna()

X_pos = df_clean.copy()
y_pos = np.ones(len(X_pos))

X_neg = df_clean.copy()
X_neg[num_cols] = -X_neg[num_cols]
y_neg = np.zeros(len(X_neg))

X = pd.concat([X_pos, X_neg], ignore_index=True)
y = np.concatenate([y_pos, y_neg])


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [17]:
preprocessor = ColumnTransformer([
    ('scale', StandardScaler(), num_cols),
    ('encode', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

pipeline = ImbPipeline([
    ('prep', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42))
])


In [19]:
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

         0.0       0.82      0.86      0.84       291
         1.0       0.87      0.83      0.85       327

    accuracy                           0.84       618
   macro avg       0.84      0.84      0.84       618
weighted avg       0.84      0.84      0.84       618

